In [53]:
!pip install plotly


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import os
from datetime import datetime
import plotly.express as px

In [55]:
#headers in small letters. figure out how to show html files
# plotly theme
pio.templates.default = "plotly_white"
pio.templates["plotly_white"].layout.update({
    'font': {'family': 'Arial, sans-serif', 'size': 12},
    'plot_bgcolor': 'white',
    'paper_bgcolor': 'white'
})

# 2. LOAD AND CLEAN DATA FROM EXACT PATH

def load_and_clean_singapore_data():   
    csv_path = r"https://raw.githubusercontent.com/Kmiriti/KPJ/refs/heads/main/data/singapore_transport_ridership.csv"
    
    try:
        # Load the CSV file
        df = pd.read_csv(csv_path)
        print(f" Successfully loaded: {len(df)} records")
        print(f" Raw data shape: {df.shape}")
        print(f" Raw columns: {list(df.columns)}")
        print(f"\n Sample data:")
        print(df.head(3))
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return None
    
    # 2. CLEAN AND STANDARDIZE COLUMN NAMES

    
    # Convert all column names to lowercase for consistency
    df.columns = df.columns.str.lower()
    
    # Common column name patterns in Singapore transport data
    column_patterns = {
        'year': ['year', '년', 'period', 'date', 'time'],
        'mode': ['mode', 'type', 'transport', 'category'],
        'ridership': ['ridership', 'value', 'count', 'number', 'average', 'daily']
    }
    
    # Map existing columns to standard names
    column_mapping = {}
    
    for standard_name, patterns in column_patterns.items():
        for col in df.columns:
            col_lower = str(col).lower()
            if any(pattern in col_lower for pattern in patterns):
                column_mapping[col] = standard_name
                print(f" Mapping: '{col}' → '{standard_name}'")
                break
    
    # Apply the mapping
    if column_mapping:
        df = df.rename(columns=column_mapping)
    else:
        print("No column mapping found. Using original column names.")
    

    
    # Ensure we have required columns
    required_columns = ['year', 'mode', 'ridership']
    
    for req_col in required_columns:
        if req_col not in df.columns:
            print(f"⚠️  Warning: '{req_col}' column not found")
            
            # Try to infer from available columns
            if req_col == 'year':
                # Look for numeric columns that could be years
                for col in df.columns:
                    if pd.api.types.is_numeric_dtype(df[col]):
                        if df[col].min() > 1900 and df[col].max() < 2100:
                            df['year'] = df[col]
                            print(f"  → Using '{col}' as year")
                            break
            
            elif req_col == 'ridership':
                # Use first numeric column for ridership
                numeric_cols = df.select_dtypes(include=[np.number]).columns
                if len(numeric_cols) > 0:
                    df['ridership'] = df[numeric_cols[0]]
                    print(f"  → Using '{numeric_cols[0]}' as ridership")
    
    # Convert data types
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['ridership'] = pd.to_numeric(df['ridership'], errors='coerce')
    
    # Clean mode names (standardize case)
    if 'mode' in df.columns:
        df['mode'] = df['mode'].str.strip().str.upper()
    
    # Remove rows with missing critical data
    initial_count = len(df)
    df = df.dropna(subset=['year', 'ridership'])
    removed_count = initial_count - len(df)
    
    # Sort by year and mode
    df = df.sort_values(['year', 'mode'])
    
    print(f"\n✅ Data cleaning complete:")
    print(f"   • Removed {removed_count} rows with missing values")
    print(f"   • Final data: {len(df)} records")
    print(f"   • Year range: {int(df['year'].min())} to {int(df['year'].max())}")
    
    if 'mode' in df.columns:
        modes = df['mode'].unique()
        print(f"   • Transport modes: {list(modes)}")
    
    return df

# Load the data
df = load_and_clean_singapore_data()

# Save cleaned version
cleaned_path = r"C:\Users\kendi\repos\KPJ\data\singapore_transport_cleaned.csv"
df.to_csv(cleaned_path, index=False, encoding='utf-8-sig')
print(f"\n💾 Cleaned data saved to: {cleaned_path}")

# ============================================
# 3. POLICY TIMELINE DEFINITION
# ============================================

print("\n" + "="*60)
print("SINGAPORE LAND TRANSPORT POLICY TIMELINE")
print("="*60)

# Singapore Land Transport Master Plans
POLICY_EVENTS = {
    1996: "First MRT Network Completion",
    2003: "Circle Line Phase 1 Opens",
    2008: "Land Transport Master Plan 2008",
    2011: "Circle Line Fully Operational",
    2013: "LTMP 2013: Walk, Cycle, Ride",
    2017: "Downtown Line Fully Operational",
    2019: "LTMP 2019: 45-Minute City",
    2022: "Thomson-East Coast Line Phase 3"
}

# Key policy years for analysis
KEY_POLICY_YEARS = [2008, 2013, 2019]

for year, event in sorted(POLICY_EVENTS.items()):
    print(f"• {year}: {event}")

# ============================================
# 4. CREATE INTERACTIVE PLOTLY VISUALIZATIONS
# ============================================

print("\n" + "="*60)
print("CREATING INTERACTIVE VISUALIZATIONS")
print("="*60)

# Define color scheme
COLOR_SCHEME = {
    'MRT': '#1F77B4',    # Blue
    'LRT': '#FF7F0E',    # Orange
    'BUSES': '#2CA02C',  # Green
    'BUS': '#2CA02C',    # Green (alternate spelling)
    'Pre-Policy': '#1F77B4',
    'Post-Policy': '#FF7F0E',
    'Policy Start': '#D62728',
    'Full Implementation': '#9467BD'
}

# ============================================
# 4.1 MAIN ANALYSIS DASHBOARD (4 PANELS)
# ============================================

def create_main_dashboard(df):
    """Create 4-panel interactive dashboard"""
    
    print("🔄 Creating 4-panel interactive dashboard...")
    
    # Create subplot figure
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '<b>A. Public Transport Ridership Trends (1995-2024)</b>',
            '<b>B. Mode Share Evolution Over Time (%)</b>',
            '<b>C. Policy Impact: Before/After Analysis</b>',
            '<b>D. Annual Growth Rates by Mode (%)</b>'
        ),
        vertical_spacing=0.15,
        horizontal_spacing=0.12,
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'bar'}, {'type': 'scatter'}]]
    )
    
    # ============================================
    # Panel A: Ridership Trends by Mode
    # ============================================
    
    modes = df['mode'].unique()
    
    for mode in modes:
        mode_data = df[df['mode'] == mode].sort_values('year')
        
        fig.add_trace(
            go.Scatter(
                x=mode_data['year'],
                y=mode_data['ridership'],
                mode='lines+markers',
                name=mode,
                line=dict(width=3),
                marker=dict(size=6),
                hovertemplate=f"<b>{mode}</b><br>" +
                             "Year: %{x}<br>" +
                             "Ridership: %{y:,.0f}<br>" +
                             "<extra></extra>",
                legendgroup=mode,
                showlegend=True
            ),
            row=1, col=1
        )
    
    # Add policy markers
    for year in KEY_POLICY_YEARS:
        fig.add_vline(
            x=year,
            line_width=2,
            line_dash="dash",
            line_color="#D62728",
            opacity=0.7,
            row=1, col=1
        )
        
        fig.add_annotation(
            x=year,
            y=0.95,
            xref="x",
            yref="paper",
            text=f"LTMP {year}",
            showarrow=False,
            font=dict(size=10, color="#D62728"),
            bgcolor="white",
            bordercolor="#D62728",
            borderwidth=1,
            borderpad=3,
            row=1, col=1
        )
    
    # ============================================
    # Panel B: Mode Share Evolution
    # ============================================
    
    # Calculate annual totals
    annual_totals = df.groupby('year')['ridership'].sum().reset_index()
    
    # Calculate mode share percentages
    mode_share_data = []
    for year in df['year'].unique():
        year_data = df[df['year'] == year]
        total = annual_totals[annual_totals['year'] == year]['ridership'].values[0]
        
        for _, row in year_data.iterrows():
            mode_share_data.append({
                'year': year,
                'mode': row['mode'],
                'share': (row['ridership'] / total * 100) if total > 0 else 0
            })
    
    mode_share_df = pd.DataFrame(mode_share_data)
    
    # Create stacked area chart
    for mode in modes:
        mode_data = mode_share_df[mode_share_df['mode'] == mode].sort_values('year')
        
        fig.add_trace(
            go.Scatter(
                x=mode_data['year'],
                y=mode_data['share'],
                mode='lines',
                name=mode,
                stackgroup='one',
                line=dict(width=0),
                fillcolor=COLOR_SCHEME.get(mode, '#888888'),
                hovertemplate=f"<b>{mode}</b><br>" +
                             "Year: %{x}<br>" +
                             "Share: %{y:.1f}%<br>" +
                             "<extra></extra>",
                legendgroup=mode,
                showlegend=False
            ),
            row=1, col=2
        )
    
    # ============================================
    # Panel C: Policy Impact Analysis
    # ============================================
    
    # Focus on MRT for policy impact analysis
    if 'MRT' in modes:
        mrt_data = df[df['mode'] == 'MRT'].sort_values('year')
        
        # Define policy periods
        policy_periods = [
            {'name': 'LTMP 2008', 'year': 2008, 'pre': 3, 'post': 5},
            {'name': 'LTMP 2013', 'year': 2013, 'pre': 3, 'post': 5},
            {'name': 'LTMP 2019', 'year': 2019, 'pre': 4, 'post': 5}
        ]
        
        policy_results = []
        for policy in policy_periods:
            policy_year = policy['year']
            
            # Get pre and post periods
            pre_start = policy_year - policy['pre']
            pre_end = policy_year - 1
            post_start = policy_year
            post_end = policy_year + policy['post']
            
            # Filter available data within these ranges
            pre_data = mrt_data[(mrt_data['year'] >= pre_start) & (mrt_data['year'] <= pre_end)]
            post_data = mrt_data[(mrt_data['year'] >= post_start) & (mrt_data['year'] <= post_end)]
            
            if len(pre_data) > 0 and len(post_data) > 0:
                pre_avg = pre_data['ridership'].mean()
                post_avg = post_data['ridership'].mean()
                growth_pct = ((post_avg - pre_avg) / pre_avg) * 100
                
                policy_results.append({
                    'policy': policy['name'],
                    'pre_avg': pre_avg,
                    'post_avg': post_avg,
                    'growth_pct': growth_pct
                })
        
        if policy_results:
            policy_df = pd.DataFrame(policy_results)
            
            # Add pre-policy bars
            fig.add_trace(
                go.Bar(
                    name='Before Policy',
                    x=policy_df['policy'],
                    y=policy_df['pre_avg'],
                    marker_color=COLOR_SCHEME['Pre-Policy'],
                    hovertemplate="<b>%{x} - Before</b><br>" +
                                 "Average: %{y:,.0f}<br>" +
                                 "<extra></extra>",
                    legendgroup='pre',
                    showlegend=True
                ),
                row=2, col=1
            )
            
            # Add post-policy bars
            fig.add_trace(
                go.Bar(
                    name='After Policy',
                    x=policy_df['policy'],
                    y=policy_df['post_avg'],
                    marker_color=COLOR_SCHEME['Post-Policy'],
                    hovertemplate="<b>%{x} - After</b><br>" +
                                 "Average: %{y:,.0f}<br>" +
                                 "Growth: %{customdata:.1f}%<br>" +
                                 "<extra></extra>",
                    customdata=policy_df['growth_pct'],
                    legendgroup='post',
                    showlegend=True
                ),
                row=2, col=1
            )
    
    # ============================================
    # Panel D: Annual Growth Rates
    # ============================================
    
    for mode in modes:
        mode_data = df[df['mode'] == mode].sort_values('year')
        
        # Calculate year-over-year growth
        mode_data = mode_data.copy()
        mode_data['growth_pct'] = mode_data['ridership'].pct_change() * 100
        
        fig.add_trace(
            go.Scatter(
                x=mode_data['year'],
                y=mode_data['growth_pct'],
                mode='lines+markers',
                name=mode,
                line=dict(width=2),
                marker=dict(size=5),
                hovertemplate=f"<b>{mode}</b><br>" +
                             "Year: %{x}<br>" +
                             "Growth: %{y:.1f}%<br>" +
                             "<extra></extra>",
                legendgroup=mode,
                showlegend=False
            ),
            row=2, col=2
        )
    
    # Add zero reference line
    fig.add_hline(
        y=0,
        line_width=1,
        line_dash="dot",
        line_color="gray",
        row=2, col=2
    )
    
    # ============================================
    # 4.2 UPDATE LAYOUT AND STYLING
    # ============================================
    
    fig.update_layout(
        title=dict(
            text="Singapore Public Transport Policy Analysis<br>Impact of Land Transport Master Plans (1995-2024)",
            font=dict(size=22, family="Arial", color="#2c3e50"),
            x=0.5,
            y=0.98
        ),
        height=1000,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor='rgba(255, 255, 255, 0.9)',
            bordercolor='rgba(0, 0, 0, 0.2)',
            borderwidth=1
        ),
        hovermode="x unified",
        plot_bgcolor='white',
        margin=dict(t=150, b=50, l=50, r=50)
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_yaxes(title_text="Ridership", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    fig.update_yaxes(title_text="Mode Share (%)", row=1, col=2)
    fig.update_xaxes(title_text="Policy", row=2, col=1)
    fig.update_yaxes(title_text="Average Ridership", row=2, col=1)
    fig.update_xaxes(title_text="Year", row=2, col=2)
    fig.update_yaxes(title_text="Growth Rate (%)", row=2, col=2)
    
    # Update subplot titles
    fig.update_annotations(font_size=14)
    
    return fig

# Create the main dashboard
main_dashboard = create_main_dashboard(df)

# ============================================
# 5. CREATE INTERACTIVE MODE COMPARISON DASHBOARD
# ============================================

def create_mode_comparison_dashboard(df):
    """Create interactive mode comparison dashboard"""
    
    print("🔄 Creating mode comparison dashboard...")
    
    fig = go.Figure()
    
    modes = df['mode'].unique()
    
    for mode in modes:
        mode_data = df[df['mode'] == mode].sort_values('year')
        
        fig.add_trace(
            go.Scatter(
                x=mode_data['year'],
                y=mode_data['ridership'],
                mode='lines+markers',
                name=mode,
                line=dict(width=3),
                marker=dict(size=8),
                hovertemplate=f"<b>{mode}</b><br>" +
                             "Year: %{x}<br>" +
                             "Ridership: %{y:,.0f}<br>" +
                             "<extra></extra>",
                visible=True
            )
        )
    
    # Create dropdown buttons for mode selection
    buttons = []
    
    # Individual mode buttons
    for i, mode in enumerate(modes):
        visibility = [False] * len(modes)
        visibility[i] = True
        
        buttons.append(
            dict(
                label=mode,
                method="update",
                args=[{"visible": visibility},
                      {"title": f"<b>{mode} Ridership Trend (1995-2024)</b>"}]
            )
        )
    
    # "All Modes" button
    buttons.append(
        dict(
            label="All Modes",
            method="update",
            args=[{"visible": [True] * len(modes)},
                  {"title": "<b>All Transport Modes: Ridership Trends (1995-2024)</b>"}]
        )
    )
    
    fig.update_layout(
        title=dict(
            text="<b>Singapore Public Transport Ridership by Mode (1995-2024)</b>",
            font=dict(size=18),
            x=0.5
        ),
        updatemenus=[
            dict(
                type="dropdown",
                direction="down",
                buttons=buttons,
                x=1.0,
                xanchor="right",
                y=1.15,
                yanchor="top",
                bgcolor="white",
                bordercolor="#2c3e50",
                borderwidth=1
            )
        ],
        xaxis_title="<b>Year</b>",
        yaxis_title="<b>Ridership</b>",
        hovermode="x unified",
        height=600,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    # Add policy markers
    for year in KEY_POLICY_YEARS:
        fig.add_vline(
            x=year,
            line_width=2,
            line_dash="dash",
            line_color="#D62728",
            opacity=0.5,
            annotation_text=f"LTMP {year}",
            annotation_position="top",
            annotation_font_size=10,
            annotation_font_color="#D62728"
        )
    
    return fig

# Create mode comparison dashboard
mode_dashboard = create_mode_comparison_dashboard(df)

# ============================================
# 6. CREATE ANNUAL SUMMARY DASHBOARD
# ============================================

def create_annual_summary_dashboard(df):
    """Create annual summary dashboard with key metrics"""
    
    print("🔄 Creating annual summary dashboard...")
    
    # Calculate annual totals and averages
    annual_summary = df.groupby('year').agg(
        total_ridership=('ridership', 'sum'),
        avg_ridership=('ridership', 'mean'),
        num_modes=('mode', 'nunique')
    ).reset_index()
    
    # Calculate year-over-year growth
    annual_summary['yoy_growth'] = annual_summary['total_ridership'].pct_change() * 100
    
    # Create figure with secondary y-axis
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '<b>Total Annual Ridership</b>',
            '<b>Year-over-Year Growth (%)</b>',
            '<b>Average Ridership per Mode</b>',
            '<b>Number of Active Modes</b>'
        ),
        vertical_spacing=0.15,
        horizontal_spacing=0.12
    )
    
    # Panel 1: Total ridership
    fig.add_trace(
        go.Bar(
            x=annual_summary['year'],
            y=annual_summary['total_ridership'],
            name='Total Ridership',
            marker_color='#1F77B4',
            hovertemplate="Year: %{x}<br>Total: %{y:,.0f}<extra></extra>"
        ),
        row=1, col=1
    )
    
    # Panel 2: YoY Growth
    fig.add_trace(
        go.Scatter(
            x=annual_summary['year'],
            y=annual_summary['yoy_growth'],
            mode='lines+markers',
            name='YoY Growth',
            line=dict(width=3, color='#FF7F0E'),
            marker=dict(size=8, color='#FF7F0E'),
            hovertemplate="Year: %{x}<br>Growth: %{y:.1f}%<extra></extra>"
        ),
        row=1, col=2
    )
    
    fig.add_hline(y=0, line_dash="dot", line_color="gray", row=1, col=2)
    
    # Panel 3: Average per mode
    fig.add_trace(
        go.Bar(
            x=annual_summary['year'],
            y=annual_summary['avg_ridership'],
            name='Avg per Mode',
            marker_color='#2CA02C',
            hovertemplate="Year: %{x}<br>Average: %{y:,.0f}<extra></extra>"
        ),
        row=2, col=1
    )
    
    # Panel 4: Number of modes
    fig.add_trace(
        go.Scatter(
            x=annual_summary['year'],
            y=annual_summary['num_modes'],
            mode='lines+markers',
            name='Active Modes',
            line=dict(width=3, color='#9467BD'),
            marker=dict(size=8, color='#9467BD', symbol='diamond'),
            hovertemplate="Year: %{x}<br>Modes: %{y}<extra></extra>"
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text="<b>Singapore Public Transport: Annual Performance Metrics</b>",
            font=dict(size=20),
            x=0.5,
            y=0.98
        ),
        height=800,
        showlegend=False,
        hovermode="x unified",
        plot_bgcolor='white'
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_yaxes(title_text="Total Ridership", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    fig.update_yaxes(title_text="Growth (%)", row=1, col=2)
    fig.update_xaxes(title_text="Year", row=2, col=1)
    fig.update_yaxes(title_text="Average Ridership", row=2, col=1)
    fig.update_xaxes(title_text="Year", row=2, col=2)
    fig.update_yaxes(title_text="Number of Modes", row=2, col=2)
    
    # Add policy markers to all panels
    for year in KEY_POLICY_YEARS:
        for row in [1, 2]:
            for col in [1, 2]:
                fig.add_vline(
                    x=year,
                    line_width=1,
                    line_dash="dash",
                    line_color="#D62728",
                    opacity=0.3,
                    row=row, col=col
                )
    
    return fig
    #fig.show() 
# Create annual summary dashboard
annual_dashboard = create_annual_summary_dashboard(df)

# 7. SAVE AND DISPLAY VISUALIZATIONS

print("\n" + "="*60)
print("SAVING VISUALIZATIONS")
print("="*60)

# Define output directory
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Save dashboards
main_dashboard.write_html(f"{output_dir}/singapore_transport_main_dashboard.html")
mode_dashboard.write_html(f"{output_dir}/singapore_transport_mode_dashboard.html")
annual_dashboard.write_html(f"{output_dir}/singapore_transport_annual_dashboard.html")

# Save mode comparison dashboard
mode_dashboard_path = os.path.join(output_dir, "singapore_transport_mode_dashboard.html")
mode_dashboard.write_html(mode_dashboard_path)
print(f"Mode dashboard saved: {mode_dashboard_path}")

# Save annual summary dashboard
annual_dashboard_path = os.path.join(output_dir, "singapore_transport_annual_dashboard.html")
annual_dashboard.write_html(annual_dashboard_path)
print(f" Annual dashboard saved: {annual_dashboard_path}")

# Save static versions for reports
try:
    main_dashboard.write_image(
        os.path.join(output_dir, "singapore_transport_main_dashboard.png"),
        width=1600, height=1000, scale=2
    )
    print(f"Static main dashboard saved as PNG")
except:
    print("Could not save static PNG (may need kaleido package)")


# 8. KEY POLICY METRICS
print("\n" + "="*60)
print("KEY POLICY METRICS AND INSIGHTS")
print("="*60)

# Calculate overall metrics
first_year = df['year'].min()
last_year = df['year'].max()

print(f"\n OVERALL PERFORMANCE ({first_year}-{last_year}):")

# Calculate total growth
for mode in df['mode'].unique():
    mode_data = df[df['mode'] == mode].sort_values('year')
    
    if len(mode_data) > 1:
        start_val = mode_data.iloc[0]['ridership']
        end_val = mode_data.iloc[-1]['ridership']
        total_growth = ((end_val - start_val) / start_val) * 100
        
        # Calculate CAGR
        years = last_year - first_year
        cagr = ((end_val / start_val) ** (1/years) - 1) * 100
        
        print(f"\n🚌 {mode}:")
        print(f"   • Start ({first_year}): {start_val:,.0f}")
        print(f"   • End ({last_year}): {end_val:,.0f}")
        print(f"   • Total Growth: {total_growth:+.1f}%")
        print(f"   • CAGR: {cagr:+.1f}%")

# Calculate policy impact for MRT
if 'MRT' in df['mode'].unique():
    mrt_data = df[df['mode'] == 'MRT'].sort_values('year')
    
    print(f"\n MRT POLICY IMPACT ANALYSIS:")
    
    for policy_year in KEY_POLICY_YEARS:
        if policy_year in mrt_data['year'].values:
            # Get 3 years before and after policy
            pre_start = policy_year - 3
            pre_end = policy_year - 1
            post_start = policy_year
            post_end = policy_year + 3
            
            pre_data = mrt_data[(mrt_data['year'] >= pre_start) & (mrt_data['year'] <= pre_end)]
            post_data = mrt_data[(mrt_data['year'] >= post_start) & (mrt_data['year'] <= post_end)]
            
            if len(pre_data) > 0 and len(post_data) > 0:
                pre_avg = pre_data['ridership'].mean()
                post_avg = post_data['ridership'].mean()
                growth = ((post_avg - pre_avg) / pre_avg) * 100
                
                print(f"   • LTMP {policy_year}:")
                print(f"     Pre-policy ({pre_start}-{pre_end}): {pre_avg:,.0f}")
                print(f"     Post-policy ({post_start}-{post_end}): {post_avg:,.0f}")
                print(f"     Growth: {growth:+.1f}%")

# Calculate mode share changes
print(f"\n MODE SHIFT ANALYSIS:")
earliest = df[df['year'] == first_year]
latest = df[df['year'] == last_year]

if len(earliest) > 0 and len(latest) > 0:
    total_early = earliest['ridership'].sum()
    total_late = latest['ridership'].sum()
    
    for mode in df['mode'].unique():
        early_mode = earliest[earliest['mode'] == mode]
        late_mode = latest[latest['mode'] == mode]
        
        if len(early_mode) > 0 and len(late_mode) > 0:
            early_share = (early_mode['ridership'].values[0] / total_early) * 100
            late_share = (late_mode['ridership'].values[0] / total_late) * 100
            share_change = late_share - early_share
            
            print(f"   • {mode}: {early_share:.1f}% → {late_share:.1f}% ({share_change:+.1f}%)")



# ============================================


'''**Output Files Created**:

1. singapore_transport_main_dashboard.html     - Comprehensive 4-panel analysis
2. singapore_transport_mode_dashboard.html     - Interactive mode comparison
3. singapore_transport_annual_dashboard.html   - Annual performance metrics
4. singapore_transport_cleaned.csv             - Cleaned dataset for further analysis
'''

 Successfully loaded: 90 records
 Raw data shape: (90, 3)
 Raw columns: ['year', 'mode', 'ridership']

 Sample data:
   year mode  ridership
0  1995  MRT     740000
1  1995  LRT          0
2  1995  Bus    3009000
 Mapping: 'year' → 'year'
 Mapping: 'mode' → 'mode'
 Mapping: 'ridership' → 'ridership'

✅ Data cleaning complete:
   • Removed 0 rows with missing values
   • Final data: 90 records
   • Year range: 1995 to 2024
   • Transport modes: ['BUS', 'LRT', 'MRT']

💾 Cleaned data saved to: C:\Users\kendi\repos\KPJ\data\singapore_transport_cleaned.csv

SINGAPORE LAND TRANSPORT POLICY TIMELINE
• 1996: First MRT Network Completion
• 2003: Circle Line Phase 1 Opens
• 2008: Land Transport Master Plan 2008
• 2011: Circle Line Fully Operational
• 2013: LTMP 2013: Walk, Cycle, Ride
• 2017: Downtown Line Fully Operational
• 2019: LTMP 2019: 45-Minute City
• 2022: Thomson-East Coast Line Phase 3

CREATING INTERACTIVE VISUALIZATIONS
🔄 Creating 4-panel interactive dashboard...
🔄 Creating mode com

'**Output Files Created**:\n\n1. singapore_transport_main_dashboard.html     - Comprehensive 4-panel analysis\n2. singapore_transport_mode_dashboard.html     - Interactive mode comparison\n3. singapore_transport_annual_dashboard.html   - Annual performance metrics\n4. singapore_transport_cleaned.csv             - Cleaned dataset for further analysis\n'